In [1]:
import emoji
import math
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.python.framework import ops
#from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict
import warnings
warnings.filterwarnings("ignore")

from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
import numpy as np
import csv
import re
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np


np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

%matplotlib inline
np.random.seed(1)

c:\users\shammi quddus\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [68]:
#Building a basic model
from keras.models import Sequential
model = Sequential()
from keras.layers import Dense

model.add(Dense(units=64, activation='relu', input_dim=300))
model.add(Dense(units=15, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

#model.compile(loss=keras.losses.categorical_crossentropy,
 #             optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True))

In [9]:
model.fit(x_train, y_train,
          epochs=100,
          batch_size=100)
score = model.evaluate(x_test, y_test, batch_size=299)
#print(score)

AttributeError: 'Word2VecKeyedVectors' object has no attribute 'fit'

# Analysis
the  minimum loss i 2.5 with an accuracy of 34.6%. Given we have five classes in this dataset, the baseline probabily of any of the five classes is 20% thus our model is doing slightly better than baseline.

In [ ]:
#Testing with our test set
tweets = []
tokenizedTweets = []
vectorizedTweets = []

with open('test_data_main.csv', 'r') as csvfile:
    textreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in textreader:
        row = row[:-1]
        row = str(row)
        row = re.sub(r'([^\s\w]|_)+', '', row)
        tweets.append(row)

model = KeyedVectors.load_word2vec_format('GoogleNews.bin', binary=True)

for index,tweet in enumerate(tweets):
    wordVector = []
    tokenizedTweets.append(tweet.split())
    for word in tokenizedTweets[index]:
        if word in model.vocab:
            wordVec = model[word]
            wordVector.append(wordVec)
    tweetVec = np.mean(wordVector,axis=0)
    vectorizedTweets.append(tweetVec)

#print (vectorizedTweets)

In [ ]:
x_test2=vectorizedTweets
np.savetxt("foo1.csv", x_test2, delimiter=",")


In [ ]:
df5=pd.read_csv('foo1.csv')
y_test2=df5.iloc[:,300:301]
y_test2.shape

# Final: Using CS230 sourcecode

In [2]:
#Splitting into random train-test set and exporting to train and test csv
#DO NOT RUN JUST GO TO NEXT CELL
df=pd.read_csv('shorter_list.csv')
msk = np.random.rand(len(df)) < 0.15
df_test = df[msk]
df_train = df[~msk]
df_train.to_csv('train_final.csv')
df_test.to_csv('test_final.csv')

In [3]:
def read_csv(filename ='emojify_data.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[1])
            emoji.append(row[2])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)
    return X, Y

In [4]:
#Reading X/Y_train, X/Y_test (remember to manually delete extra column from csv files)
X_train, Y_train = read_csv('train_final.csv')
X_test, Y_test = read_csv('test_final.csv')
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((5774,), (1024,), (5774,), (1024,))

In [5]:
#Getting maxlength of X
maxLen = len(max(X_train, key=len).split())
print(maxLen)

21


In [6]:
import csv
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [7]:
#Getting three dictionaries
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.txt')

In [8]:
#Testing dictionary
word = "cucumber"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

the index of cucumber in the vocabulary is 113317
the 289846th word in the vocabulary is potatos


In [9]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = (X[i].lower()).split()
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            try:
                X_indices[i, j] = word_to_index[w]
                # Increment j to j + 1
                j = j+1
            except:
                pass    

    ### END CODE HERE ###
    
    return X_indices

In [10]:
#Testing if word to index works, so that all examples are of same length
print(word_to_index['funny'])
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 6)
print("X1 =", X1)
print("X1_indices =", X1_indices)

155345
X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[155345. 225122.      0.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.      0.]]


In [11]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [12]:
#Converting to X_train_indices and one-hot Y_train matrix
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 150)
#Converting to X_test_indices and one-hot Y_test matrix
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 150)

In [13]:
#Checking shapes
X_train_indices.shape, Y_train.shape, Y_train_oh.shape

((5774, 21), (5774,), (5774, 150))

# Baseline with a simple Dense layer

In [14]:
#Building a simple neural net with one layer, basically a logistic regression

from keras.models import Sequential
from keras.layers import Dense

model_baseline = Sequential()
model_baseline.add(Dense(150, activation='relu', input_dim=maxLen))
model_baseline.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model_baseline.fit(X_train_indices, Y_train, epochs=50, batch_size=100)
loss, acc = model_baseline.evaluate(X_test_indices, Y_test,batch_size=100)
print("Test accuracy = ", acc)

Epoch 1/50
5774/5774 [==============================] - 1s 145us/step - loss: 12.8530 - acc: 0.0054
Epoch 2/50
5774/5774 [==============================] - 0s 27us/step - loss: 12.8542 - acc: 0.0054
Epoch 3/50
5774/5774 [==============================] - 0s 28us/step - loss: 12.8542 - acc: 0.0054
Epoch 4/50
5774/5774 [==============================] - 0s 29us/step - loss: 12.8542 - acc: 0.0054
Epoch 5/50
5774/5774 [==============================] - 0s 28us/step - loss: 12.8542 - acc: 0.0054
Epoch 6/50
5774/5774 [==============================] - 0s 27us/step - loss: 12.8542 - acc: 0.0054
Epoch 7/50
5774/5774 [==============================] - 0s 28us/step - loss: 12.8542 - acc: 0.0054
Epoch 8/50
5774/5774 [==============================] - 0s 30us/step - loss: 12.8542 - acc: 0.0054
Epoch 9/50
5774/5774 [==============================] - 0s 30us/step - loss: 12.8542 - acc: 0.0054
Epoch 10/50
5774/5774 [==============================] - 0s 29us/step - loss: 12.8542 - acc: 0.0054
Epoch 11

## Performance of baseline model
The accuracy of both the train and test models are only 0.74% and 0.603%. There is slight overfitting, but that is hardly a concern given our accuracy is so low.

# Building a bidirectional LSTM model

In [71]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                 # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [72]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape= input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index) 
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings =  embedding_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X =  Bidirectional(LSTM(128, activation='tanh', return_sequences=True, recurrent_activation='hard_sigmoid'))(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5, noise_shape=None, seed=None)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = Bidirectional(LSTM(128, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True))(X)
    # Add dropout with a probability of 0.5
    X =  Dropout(0.5, noise_shape=None, seed=None)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(150, activation='softmax', use_bias=True)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model =  Model(inputs = [sentence_indices], outputs = [X] , name = 'LSTMmodel')
    
    ### END CODE HERE ###
    
    return model

In [73]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

ValueError: could not broadcast input array from shape (49) into shape (50)

In [56]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [65]:
model.fit(X_train_indices, Y_train_oh, epochs = 20, batch_size = 200, shuffle=True)

Epoch 1/20
5804/5804 [==============================] - 22s 4ms/step - loss: 4.9196 - acc: 0.1044
Epoch 2/20
5804/5804 [==============================] - 23s 4ms/step - loss: 4.9185 - acc: 0.1042
Epoch 3/20
5804/5804 [==============================] - 23s 4ms/step - loss: 4.9148 - acc: 0.1089
Epoch 4/20
5804/5804 [==============================] - 23s 4ms/step - loss: 4.9122 - acc: 0.1099
Epoch 5/20
5804/5804 [==============================] - 23s 4ms/step - loss: 4.9115 - acc: 0.1111
Epoch 6/20
5804/5804 [==============================] - 23s 4ms/step - loss: 4.9105 - acc: 0.1115
Epoch 7/20
5804/5804 [==============================] - 23s 4ms/step - loss: 4.9085 - acc: 0.1137
Epoch 8/20
5804/5804 [==============================] - 23s 4ms/step - loss: 4.9075 - acc: 0.1144
Epoch 9/20
5804/5804 [==============================] - 23s 4ms/step - loss: 4.9057 - acc: 0.1168
Epoch 10/20
5804/5804 [==============================] - 24s 4ms/step - loss: 4.9044 - acc: 0.1184
Epoch 11/20
5804/58

In [58]:
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

994/994 [==============================] - 2s 2ms/step

Test accuracy =  0.11066398390342053


# Results
- with 2 bidirectional LSTM + dense + softmax accuracy is 29.46% (test accuracy is 29.08% so no overfitting)
- no change with droput, makes sense since no overfitting really.

In [60]:
#Twitter Dictionary containing 134 unique emojis
with open('short_dict_150.csv', mode='r') as infile:
    reader = csv.reader(infile)
    our_dict= dict(reader)

In [61]:
#Function to read emoji
def label_to_emoji(label):
    """
    Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
    """
    output=[]
    for i in label:
        out= emoji.emojize(our_dict[str(i)], use_aliases=True)
        output.append(out)
    return output

In [62]:
#Choosing top five emojis
def top_five(arr):
    top_five=arr.argsort()[-5:][::-1]
    return top_five

In [435]:
#C = 138
#y_test_oh = np.eye(C)[Y_test.reshape(-1)]
#X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        #print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

SyntaxError: unexpected EOF while parsing (<ipython-input-435-cfee3431887f>, line 9)

In [63]:
#X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
pred
#X_test_indices

array([[0.00709841, 0.01663896, 0.0065963 , ..., 0.0065963 , 0.0065963 ,
        0.0065963 ],
       [0.01550558, 0.00762046, 0.00660042, ..., 0.00660041, 0.00660041,
        0.00660042],
       [0.01053756, 0.01122721, 0.00660955, ..., 0.00660954, 0.00660954,
        0.00660954],
       ...,
       [0.01068805, 0.01107317, 0.00660958, ..., 0.00660957, 0.00660957,
        0.00660957],
       [0.01772891, 0.00665946, 0.00659196, ..., 0.00659196, 0.00659196,
        0.00659196],
       [0.01399982, 0.00844557, 0.00660498, ..., 0.00660498, 0.00660497,
        0.00660498]], dtype=float32)

In [64]:
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = top_five(pred[i])
    print(X_test[i])
    print(label_to_emoji(num))

Missing you two gutted youre not here with us love you both see you next week   

['✨', '❤', '♀', '♂', '➡']
Rancho Del Rey III Dusclops  til 23285126m 52s  

['❤', '✨', '♀', '♂', '♥']
Im more of like a  yes nigga damn   

['✨', '❤', '♀', '♂', '♥']
MustFollow  bambiblacks gt Visit her website here   And be a member now  

['✨', '❤', '♀', '♂', '➡']
MustFollow  bambiblacks gt Visit her website here   And be a member now  

['✨', '❤', '♀', '♂', '➡']
Alright recovery was cool I ate A LOT but Im fat and over it time to fast for 4 days  

['✨', '❤', '♀', '♂', '➡']
Serving incredible looks all Halloween   

['✨', '❤', '♀', '♂', '♥']
JBJofficial787   

['✨', '❤', '♀', '♂', '➡']
I organized the threads into a hopefully easy to browse Moment Cats Creepy Nature Threads For October 

['✨', '❤', '♀', '♂', '♥']
Doggie conga line  donateaphoto jnj donateaphoto AIDSFreeGeneration makeHIVHistory GCFest  

['❤', '✨', '♀', '♂', '♥']
Too cute FutureLEOs FuturePolice ServeAndProtect GodBlessOurLEOs   

['❤'


['❤', '✨', '♀', '♂', '♥']
Boogie1969 BabesPromo2 AdultBrazil HoneyB69 DoujinsApp DRM00RE TumejorSonris Natuky85 PawgWithaBlog  PornoAn 

['✨', '❤', '♀', '♂', '➡']
NastyCSA AreeceSA NastyCSA Boity Boitythulo Hot sauce ma nigga  

['❤', '✨', '♀', '♂', '♥']
u aint got shit to say to me i aint got shit to say to u  

['✨', '❤', '♀', '♂', '➡']
ParaSport  Get your tickets for the BlindSports Hall of Fame Gala today  

['✨', '❤', '♀', '♂', '➡']
syajama bro  this is bad and its getting worse 

['✨', '❤', '♀', '♂', '➡']
Spurs are WRECKING Real Madrid  

['✨', '❤', '♀', '♂', '♥']
The latest example of the proven pathway NA3HL  NAHL  NCAA  

['✨', '❤', '♀', '♂', '➡']
Went from 300 contacts to 100  

['✨', '❤', '♀', '♂', '➡']
 VIDEO  Liverpool Vs Maribor Video of Liverpool 2  0 Maribor  Emre Can 

['✨', '❤', '♀', '♂', '➡']
 Discover and Engage  SLIM Enterprises  Animation  Software WebDesign      

['✨', '❤', '♀', '♂', '➡']
 Sevilla x Spartak Moscow UEFA Champions League Grupo E Ramn Snchez Pizju